## Import Dependencies

In [105]:
import petpy
from petpy import Petfinder
from config import API_key, API_secret
import json
import pymongo
import pandas as pd
from pprint import pprint

## API Call and Pandas DF Processing

In [106]:
pf = Petfinder(key=API_key, secret=API_secret)

In [143]:
# animals = pf.animals() - see below for query with edited results limits
animal_df = pf.animals(results_per_page=100, pages=100, return_df=True)

C:\Users\kaldm\anaconda3\envs\PythonData\lib\site-packages\petpy\api.py:1049: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = json_normalize(results[key])


In [211]:
animal2_df = animal_df.copy()

In [213]:
# get rid of extra organization_id column
my_columns = list(range(0,48))
animal2_df = animal2_df.iloc[:,my_columns]

In [230]:
# inspect list for column name formats
column_list = list(animal2_df.columns) 
column_list

['id',
 'organization_id',
 'url',
 'type',
 'species',
 'age',
 'gender',
 'size',
 'coat',
 'tags',
 'name',
 'description',
 'organization_animal_id',
 'photos',
 'videos',
 'status',
 'status_changed_at',
 'published_at',
 'distance',
 'breeds.primary',
 'breeds.secondary',
 'breeds.mixed',
 'breeds.unknown',
 'colors.primary',
 'colors.secondary',
 'colors.tertiary',
 'attributes.spayed_neutered',
 'attributes.house_trained',
 'attributes.declawed',
 'attributes.special_needs',
 'attributes.shots_current',
 'environment.children',
 'environment.dogs',
 'environment.cats',
 'primary_photo_cropped.small',
 'primary_photo_cropped.medium',
 'primary_photo_cropped.large',
 'primary_photo_cropped.full',
 'contact.email',
 'contact.phone',
 'contact.address.address1',
 'contact.address.address2',
 'contact.address.city',
 'contact.address.state',
 'contact.address.postcode',
 'contact.address.country',
 'animal_id',
 'animal_type']

In [232]:
# change column header names in animal2_df to remove dot notation for mongo
animal3_df = animal2_df.rename(columns = {
 'breeds.primary':'breeds_primary',
 'breeds.secondary': 'breeds_secondary',
 'breeds.mixed': 'breeds_mixed',
 'breeds.unknown':'breeds_unknown',
 'colors.primary': 'colors_primary',
 'colors.secondary': 'colors_secondary',
 'colors.tertiary': 'colors_tertiary',
 'attributes.spayed_neutered': 'attributes_spayed_neutered',
 'attributes.house_trained': 'attributes_house_trained',
 'attributes.declawed': 'attributes_declawed',
 'attributes.special_needs': 'attributes_special_needs',
 'attributes.shots_current': 'attributes_shots_current',
 'environment.children': 'environment_children',
 'environment.dogs': 'environment_dogs',
 'environment.cats': 'environment_cats',
 'primary_photo_cropped.small': 'primary_photo_cropped_small',
 'primary_photo_cropped.medium': 'primary_photo_cropped_medium',
 'primary_photo_cropped.large': 'primary_photo_cropped_large',
 'primary_photo_cropped.full': 'primary_photo_cropped_full',
 'contact.email': 'contact_email',
 'contact.phone': 'contact_phone',
 'contact.address.address1': 'contact_address_address1',
 'contact.address.address2': 'contact_address_address2',
 'contact.address.city': 'contact_address_city',
 'contact.address.state': 'contact_address_state',
 'contact.address.postcode': 'contact_address_postcode',
 'contact.address.country': 'contact_address_country'})

In [233]:
animal3_df.head()

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact_email,contact_phone,contact_address_address1,contact_address_address2,contact_address_city,contact_address_state,contact_address_postcode,contact_address_country,animal_id,animal_type
0,50374010,CA1080,https://www.petfinder.com/dog/erin-50374010/ca...,Dog,Dog,Baby,Female,Small,Short,[],...,anitaboeker@gmail.com,None,None,None,San Diego,CA,92102,US,50374010,dog
1,50374005,OK37,https://www.petfinder.com/dog/steve-352136-503...,Dog,Dog,Adult,Male,Medium,None,[],...,animalwelfare@okc.gov,405-297-3100,2811 SE 29th Street,None,Oklahoma City,OK,73129,US,50374005,dog
2,50374002,NY1420,https://www.petfinder.com/cat/henry-super-swee...,Cat,Cat,Young,Male,Medium,Short,[],...,info@advocatrescue.org,None,None,None,Brooklyn,NY,11226,US,50374002,cat
3,50373939,FL1695,https://www.petfinder.com/cat/toasty-50373939/...,Cat,Cat,Baby,Female,Small,Short,"[Loving, playful, kitten, siamese, rescue]",...,loveisferal@gmail.com,(954) 667-9633,None,None,Margate,FL,33063,US,50373939,cat
4,50373990,IL905,https://www.petfinder.com/dog/frost-50373990/i...,Dog,Dog,Senior,Male,Small,Short,"[Loving, Cuddly, Deaf, Blind]",...,Inquiries@tinyntallrescue.com,None,None,None,St. Charles,IL,60175,US,50373990,dog


In [5]:
organizations_df = pf.organizations(results_per_page=100, pages=100, return_df=True)

C:\Users\kaldm\anaconda3\envs\PythonData\lib\site-packages\petpy\api.py:1049: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = json_normalize(results[key])


In [13]:
# photos field is creating an error in pandas because it contains a list within the field. 
org_df = organizations_df.drop(columns=['photos'])

In [25]:
# change column header names to remove dot notation for mongo
org2_df = org_df.rename(columns = {
 'address.address1': "address_address1",
 'address.address2': "address_address2",
 'address.city': "address_city" ,
 'address.state': "address_state",
 'address.postcode': "address_postcode",
 'address.country': "address_country",
 'hours.monday': "hours_monday",
 'hours.tuesday': "hours_tuesday",
 'hours.wednesday': "hours_wednesday",
 'hours.thursday': "hours_thursday",
 'hours.friday': "hours_friday",
 'hours.saturday': "hours_saturday",
 'hours.sunday': "hours_sunday",
 'adoption.policy': "adoption_policy" ,
 'adoption.url': "adoption_url",
 'social_media.facebook': "socialmedia_facebook",
 'social_media.twitter': "socialmedia_twitter",
 'social_media.youtube': "socialmedia_youtube",
 'social_media.instagram': "socialmedia_instagram",
 'social_media.pinterest': "socialmedia_pinterest",
})

In [26]:
# now try to drop duplicates - based on all columns
org2_df.drop_duplicates()
# new length is 8932

,id,name,email,phone,url,website,mission_statement,distance,address_address1,address_address2,...,hours_saturday,hours_sunday,adoption_policy,adoption_url,socialmedia_facebook,socialmedia_twitter,socialmedia_youtube,socialmedia_instagram,socialmedia_pinterest,organization_id
0,PA912,The Shelter Animal Project,tsapadoption@gmail.com,None,https://www.petfinder.com/member/us/pa/fairles...,None,None,None,PO Box 24,None,...,None,None,None,None,None,None,None,None,None,pa912
1,GA762,Special Touch Animal Rescue,STARRadopt@aol.com,(404) 983-3581,https://www.petfinder.com/member/us/ga/acworth...,None,We are a private foster home group. We are NOT...,None,None,None,...,None,None,None,None,None,None,None,None,None,ga762
2,TX876,Gina's Heart of Gold Reptile Rescue,gdisteldorf@comcast.net,(713) 329-9304,https://www.petfinder.com/member/us/tx/houston...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,tx876
3,OH749,Purr-fect Companions Sanctuary,purrfectcompanions@att.net,216-671-6369,https://www.petfinder.com/member/us/oh/lakewoo...,None,None,None,P.O.Box 770992,None,...,None,None,None,None,None,None,None,None,None,oh749
4,MO556,Montgomery City Animal Shelter/Project Preciou...,projectpreciouspaws@hotmail.com,1-573-310-4979,https://www.petfinder.com/member/us/mo/montgom...,None,None,None,915 Benton Street,None,...,None,None,None,None,None,None,None,None,None,mo556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,BC229,L.E.A.S.H. Animal Welfare Society,connect@leashsavinglives.com,,https://www.petfinder.com/member/ca/bc/pitt-me...,None,Located in Beautiful British Columbia and serv...,None,PO BOX 6,None,...,None,None,"Unless otherwise stated, the standard adoption...",None,None,None,None,None,None,bc229
9996,NY1032,Empire Animal Rescue Society,empirerescue@gmail.com,None,https://www.petfinder.com/member/us/ny/salaman...,None,None,None,None,None,...,None,None,None,https://dc6ed6c6-85fc-4cc9-93e8-8a3b736c936c.f...,None,None,None,None,None,ny1032
9997,CA2835,510 Paws Rescue,rescuedoggies510@rocketmail.com,(510) 555-0000,https://www.petfinder.com/member/us/ca/oakland...,None,We rescue and save as many dogs as we can.,None,None,None,...,None,None,None,None,https://www.facebook.com/510pawsrescue,None,None,None,None,ca2835
9998,FL1349,Fur The Love Of Animals,ftloa.rescue@gmail.com,,https://www.petfinder.com/member/us/fl/orlando...,None,Adoptions are $75.00 per cat. If you adopt 2 c...,None,None,None,...,None,None,2 cats for $90.00 and 3 cats for $100.00.\n\nI...,None,None,None,None,None,None,fl1349


## Lat and Long DF and Add to Org Collection

In [64]:
zipfile_df = pd.read_csv("col_resources/zip_lat_lon.csv")

In [65]:
zipcode_df = zipfile_df[["Zip", "Latitude", "Longitude"]]

In [66]:
#need to convert datatypes in order to merge dataframes 
convert_zipdict = {'Zip': str}
zip_converted_df = zipcode_df.astype(convert_zipdict)

In [67]:
#need to convert datatypes in order to merge dataframes
convert_orgdict = {'address_postcode': str}
org_converted_df = org2_df.astype(convert_orgdict)

In [68]:
#rename field in zipcode to merge with org df 
new_zip_df = zip_converted_df.rename(columns={'Zip': 'address_postcode'})

In [69]:
#merge zip code and org dataframes - now every org should have a corresponding latlong
org_zip_df = pd.merge(new_zip_df, org_converted_df,  how= "inner", on=["address_postcode", "address_postcode"])

In [70]:
org_zip2_df = org_zip_df.drop(columns=['organization_id'])

In [71]:
# rename id column (with CAPS) as organization_id column
org_zip3_df = org_zip2_df.rename(columns={'id': 'organization_id'})

In [234]:
# convert datatypes in order to merge dataframes 
convert_animaldict = {'id': float}
animal_conv_df = animal3_df.astype(convert_animaldict)

In [235]:
animal_conv_df.head()

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact_email,contact_phone,contact_address_address1,contact_address_address2,contact_address_city,contact_address_state,contact_address_postcode,contact_address_country,animal_id,animal_type
0,50374010.0,CA1080,https://www.petfinder.com/dog/erin-50374010/ca...,Dog,Dog,Baby,Female,Small,Short,[],...,anitaboeker@gmail.com,None,None,None,San Diego,CA,92102,US,50374010,dog
1,50374005.0,OK37,https://www.petfinder.com/dog/steve-352136-503...,Dog,Dog,Adult,Male,Medium,None,[],...,animalwelfare@okc.gov,405-297-3100,2811 SE 29th Street,None,Oklahoma City,OK,73129,US,50374005,dog
2,50374002.0,NY1420,https://www.petfinder.com/cat/henry-super-swee...,Cat,Cat,Young,Male,Medium,Short,[],...,info@advocatrescue.org,None,None,None,Brooklyn,NY,11226,US,50374002,cat
3,50373939.0,FL1695,https://www.petfinder.com/cat/toasty-50373939/...,Cat,Cat,Baby,Female,Small,Short,"[Loving, playful, kitten, siamese, rescue]",...,loveisferal@gmail.com,(954) 667-9633,None,None,Margate,FL,33063,US,50373939,cat
4,50373990.0,IL905,https://www.petfinder.com/dog/frost-50373990/i...,Dog,Dog,Senior,Male,Small,Short,"[Loving, Cuddly, Deaf, Blind]",...,Inquiries@tinyntallrescue.com,None,None,None,St. Charles,IL,60175,US,50373990,dog


## Merge DFs to One and Send to MongoDB

In [236]:
# all_df = pd.merge(animal_df, org2_df,  how= "inner", on=["organization_id", "organization_id"])
all_df = pd.merge(animal_conv_df, org_zip3_df, on=["organization_id", "organization_id"])

In [237]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [238]:
db = client.rescue_angels_db

In [239]:
final_mongo_dict = all_df.to_dict("records")

In [240]:
db.final_data.insert_many(final_mongo_dict)